In [ ]:
# import packages
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# set database connection
HOST = "10.10.2.10"
DB = "appliedda"
sql_conn = create_engine("postgresql://{}/{}".format(HOST, DB))

In [ ]:
# get columns from some table #

# table name
tbl = "il_des_establishment"
# will also later need what schema table is in:
tbl_schema = 'ides'

# the information schema is useful.
query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{}';".format(tbl)

# if you want you can test this in the SQL interface of your choosing
print(query)

In [ ]:
df_columns = pd.read_sql(query, sql_conn)
df_columns

In [ ]:
# we're going to use SQL to calculate some descriptive stats on only the numeric columns, so first subset the above
df_columns = df_columns[df_columns['data_type'].isin(['bigint', 'double precision'])]
df_columns

In [ ]:
# obviously this is for demonstrative purposes only as the full descriptive stats of this particular table 
# won't be all that interesting, but we can loop through some SQL aggregations as well as the columns

# SQL aggegation functions to use:
sql_aggs = ['COUNT', 'AVG', 'SUM', 'MIN', 'MAX', 'STDDEV']

# set the beggining of your query
sql_query = "SELECT "

# loop first through columns
for idx, row in df_columns.iterrows():
    # then iterate through the aggregation functions
    for a in sql_aggs:
        # and add the appropriate string parts to your existing string
        # here we're making use of the format function to both
        # set up the aggregation with parenthesis eg "COUNT(index)" and 
        # rename the output to <column>_<agg> eg "index_COUNT"
        sql_query += "{0:}({1:}) AS {1:}_{0:}, ".format(a, row['column_name'])
        
# pause there and see what we have
print(sql_query)

In [ ]:
# almost perfect, except since we set it up to but a comma and space after each iteration, we have an extra comma
# so let's simply chop it off by selecting everything except the last two characters
sql_query = sql_query[:-2]
sql_query

In [ ]:
# that's the select part, now let's simply add on the FROM:
sql_query += " FROM {}.{};".format(tbl_schema, tbl)

In [ ]:
# check the query - and if you want run this in some other place:
sql_query

In [ ]:
# and run the query:
summary_stats = pd.read_sql(sql_query, sql_conn)
summary_stats